In [1]:
import geopandas as gpd
import numpy as np
import folium
from folium import IFrame
import matplotlib.pyplot as plt
import os
import pandas as pd
import base64
import math

In [2]:
df=pd.read_csv('full_dataset.csv',sep=',')

In [3]:
df

,panoid,Latitude,Longitude,date,angle,type
0,cLZitfFmaUOoNaHACktEgA,-1.006709,34.170750,2021-11,0,non_crop
1,cLZitfFmaUOoNaHACktEgA,-1.006709,34.170750,2021-11,90,non_crop
2,cLZitfFmaUOoNaHACktEgA,-1.006709,34.170750,2021-11,180,non_crop
3,cLZitfFmaUOoNaHACktEgA,-1.006709,34.170750,2021-11,270,crop
4,3bZfZ_BP1QQOqX2Dcp8JOw,-1.006619,34.171045,2021-11,0,non_crop
...,...,...,...,...,...,...
13016,kZVvB-6GQSOwfV3d2tIf6Q,-0.945474,37.822005,2008-10,270,undecided
13017,lZYDVsNi80Wp7w-luvAgvQ,-3.506178,38.377727,2012-07,270,undecided
13018,I2-dUfVUZSXFpBxO5bwUqw,-4.070773,38.737468,2008-12,180,undecided
13019,vpl1H5zLr7FHWFVXNCBRgA,-0.497100,39.503369,2019-04,270,undecided


In [4]:
def triangle(lat,lon,angle,distance_m=70):
    
    distance_degrees = distance_m/(40075000 / 360)
    if angle==0: # Face to North -> need points North Est and North Ouest
        
        # Calculate new coordinates North Est
        latitude1 = lat + distance_degrees * math.cos(math.radians(45))
        longitude1 = lon + distance_degrees * math.sin(math.radians(45))
        # Calcul des nouvelles coordonnées North Ouest
        latitude2 = lat + distance_degrees * math.cos(math.radians(-45))
        longitude2 = lon + distance_degrees * math.sin(math.radians(-45))
        
    if angle==90: # Face to Est -> need points North Est and South Est
        
        # Calculate new coordinates North Est
        latitude1 = lat + distance_degrees * math.cos(math.radians(45))
        longitude1 = lon + distance_degrees * math.sin(math.radians(45))
        # Calculate new coordinates South Est
        latitude2 = lat - distance_degrees * math.cos(math.radians(45))
        longitude2 = lon + distance_degrees * math.sin(math.radians(45))
        
    if angle==180: # Face to South -> need points South Est and South Ouest
        
        # Calculate new coordinates South Est
        latitude1 = lat - distance_degrees * math.cos(math.radians(45))
        longitude1 = lon + distance_degrees * math.sin(math.radians(45))
        # Calculate new coordinates South Ouest
        latitude2 = lat - distance_degrees * math.cos(math.radians(-45))
        longitude2 = lon + distance_degrees * math.sin(math.radians(-45))
        
    if angle==270: # Face to Oeust -> need points North Ouest and South Ouest
        
        # Calculate new coordinates North Ouest
        latitude1 = lat + distance_degrees * math.cos(math.radians(-45))
        longitude1 = lon + distance_degrees * math.sin(math.radians(-45))
        # Calculate new coordinates South Ouest
        latitude2 = lat - distance_degrees * math.cos(math.radians(-45))
        longitude2 = lon + distance_degrees * math.sin(math.radians(-45))
        
    return([latitude1,longitude1,latitude2,longitude2])

In [1]:
# Uncomment the following section to see the interactive map
'''

data=df.copy()

# image folder path
image_dir = 'full_dataset/crop/'

# Create map
map = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=7)


for i in range(len(data)):
    im = data.iloc[i]
    lat = im['Latitude']
    long = im['Longitude']
    label = im['type']

    # Add points for every crops
    if label == 'crop':
        # Create full image path
        image_filename = im['panoid']+str(im['angle'])+'.jpg'
        image_path = os.path.join(image_dir, image_filename)

        # Read the image as an array of bytes
        with open(image_path, 'rb') as f:
            image_bytes = f.read()

        # Convert image to base64 format
        image_base64 = base64.b64encode(image_bytes).decode()

        # Create HTML code for popup with embedded image
        popup_html = '<img src="data:image/jpeg;base64,{}" width="300" height="300">'.format(image_base64)
        
        pts=triangle(im['Latitude'],im['Longitude'],im['angle'])
        point1 = (im['Latitude'],im['Longitude'])  
        point2 = (pts[0], pts[1])
        point3 = (pts[2], pts[3])  
        
        # Add marker with popup
        folium.CircleMarker([lat, long], radius=3, color='green', fill=True, fill_color='green',
                            popup=folium.Popup(popup_html, max_width=800)).add_to(map)
        # Add a line connecting the points of the triangle
        folium.PolyLine(locations=[point1, point2, point3, point1], color='black').add_to(map)
   
    # Add non crop images in black point 
    if label == 'non_crop':
        folium.CircleMarker([lat, long], radius=2, color='black', fill=True, fill_color='black', popup=label).add_to(map)
   
    # Add undecided images in red 
    if label == 'undecided':
        folium.CircleMarker([lat, long], radius=2, color='red', fill=True, fill_color='red', popup=label).add_to(map)

    
    
# Show map
map


'''


'\n\ndata=df.copy()\n\n# image folder path\nimage_dir = \'full_dataset/crop/\'\n\n# Create map\nmap = folium.Map(location=[data[\'Latitude\'].mean(), data[\'Longitude\'].mean()], zoom_start=7)\n\n\nfor i in range(len(data)):\n    im = data.iloc[i]\n    lat = im[\'Latitude\']\n    long = im[\'Longitude\']\n    label = im[\'type\']\n\n    # Add points for every crops\n    if label == \'crop\':\n        # Create full image path\n        image_filename = im[\'panoid\']+str(im[\'angle\'])+\'.jpg\'\n        image_path = os.path.join(image_dir, image_filename)\n\n        # Read the image as an array of bytes\n        with open(image_path, \'rb\') as f:\n            image_bytes = f.read()\n\n        # Convert image to base64 format\n        image_base64 = base64.b64encode(image_bytes).decode()\n\n        # Create HTML code for popup with embedded image\n        popup_html = \'<img src="data:image/jpeg;base64,{}" width="300" height="300">\'.format(image_base64)\n        \n        pts=triangle(i

In [ ]:
# Save the triangle field of view shapefiles
import geopandas as gpd
from shapely.geometry import Polygon

triangle_folder='labelled_triangle_nigeria/'

In [8]:


l_tri=[]

for i in range(len(data)):
    im = data.iloc[i]
    lat = im['Latitude']
    long = im['Longitude']
    label = im['type']
    if label == 'crop':
        date=im['date']
        pts=triangle(im['Latitude'],im['Longitude'],im['angle'])
        point1 = (im['Longitude'],im['Latitude'])  
        point2 = (pts[1], pts[0])
        point3 = (pts[3], pts[2])  
        # Creating triangle geometry
        triangle_polygon = Polygon([point1,point2,point3])

        # Creating the GeoPandas DataFrame with the triangle
        triangle_gdf = gpd.GeoDataFrame(geometry=[triangle_polygon])

        # Definition of the triangle's reference coordinate system (CRS)
        triangle_gdf.crs = "EPSG:4326"  # WGS84

        # Save triangle as Shapefile
        output_path = triangle_folder+im['panoid']+'_'+str(im['angle'])+'_'+date+'.shp'
        triangle_gdf.to_file(output_path)
